# NER and Sentiment

In this section we will work through applying basic sentiment analysis to our data using a pre-built *distilBERT* model from the **Flair** library. We will then use our organization labels captured through NER in the previous section to create a list of organizations with the highest and lowest average sentiment scores.

In [1]:
import pandas as pd
import flair

We initialize the English sentiment model `en-sentiment`:

In [2]:
model = flair.models.TextClassifier.load('en-sentiment')

2021-02-27 12:23:37,980 loading file C:\Users\James\.flair\models\sentiment-en-mix-distillbert_3.1.pt


For each sample there are a few steps we need to take to create the sentiment score. We need to tokenize the input text, make a prediction, extract the direction (*positive* or *negative*) and confidence (a score from 0 to 1). If this is new to you, we cover the Flair sentiment model in more depth in **TK insert link**.

The following function carries out each of these steps for a single extract:

In [18]:
def get_sentiment(text):
    # tokenize input text
    sentence = flair.data.Sentence(text)
    # make sentiment prediction
    model.predict(sentence)
    # extract sentiment direction and confidence (label and score) object
    sentiment = sentence.labels[0]
    return sentiment

We now need to load our previously processed dataframe (which includes the *organizations* column) and `apply` the `get_sentiment` function to the *selftext* column. These sentiment scores will then be stored in a new *sentiment* column.

In [4]:
# load data
df = pd.read_csv('./data/processed/reddit_investing_ner.csv', sep='|')
df.head()

,created_utc,downs,id,score,selftext,subreddit,title,ups,upvote_ratio,organizations
0,1.614290e+09,0.0,t3_lshtjn,10.0,Bloomberg article: [https://www.bloomberg.com/...,investing,Fed Views Rising Yields as Bullish Sign Reflec...,10.0,0.86,"['Bostic', 'St. Louis Fed', 'Raphael Bostic', ..."
1,1.614286e+09,0.0,t3_lsgahw,56.0,Given the recent downturn in stocks especially...,investing,ARK ETFs implosion risk ------------------------,56.0,0.83,['ARK']
2,1.614283e+09,0.0,t3_lsf8td,1.0,[https://twitter.com/desogames/status/13649710...,investing,The Counter-Party Risk Bubble,1.0,0.53,"['ITM', 'OWN', 'Citadel']"
3,1.614282e+09,0.0,t3_lsf3nh,6.0,"When you think of futures, what comes to your ...",investing,Futures were made for days like these,6.0,0.62,['NQ']
4,1.614278e+09,0.0,t3_lsdcib,3.0,I've been on this sub for quite some time and ...,investing,Let's talk about liquidity premiums,3.0,0.67,[]


In [19]:
# get sentiment
df['sentiment'] = df['selftext'].apply(get_sentiment)
df.head()

,created_utc,downs,id,score,selftext,subreddit,title,ups,upvote_ratio,organizations,sentiment
0,1.614290e+09,0.0,t3_lshtjn,10.0,Bloomberg article: [https://www.bloomberg.com/...,investing,Fed Views Rising Yields as Bullish Sign Reflec...,10.0,0.86,"[Bostic, St. Louis Fed, Raphael Bostic, the At...",NEGATIVE (0.9916)
1,1.614286e+09,0.0,t3_lsgahw,56.0,Given the recent downturn in stocks especially...,investing,ARK ETFs implosion risk ------------------------,56.0,0.83,[ARK],NEGATIVE (0.9975)
2,1.614283e+09,0.0,t3_lsf8td,1.0,[https://twitter.com/desogames/status/13649710...,investing,The Counter-Party Risk Bubble,1.0,0.53,"[ITM, OWN, Citadel]",NEGATIVE (0.9996)
3,1.614282e+09,0.0,t3_lsf3nh,6.0,"When you think of futures, what comes to your ...",investing,Futures were made for days like these,6.0,0.62,[NQ],NEGATIVE (0.9999)
4,1.614278e+09,0.0,t3_lsdcib,3.0,I've been on this sub for quite some time and ...,investing,Let's talk about liquidity premiums,3.0,0.67,[],NEGATIVE (0.9893)


Now we need to extract each of the organizations alongside it's sentiment score. We will then loop through each, tallying up a total sentiment score and count.

Before we do that, we need to convert each value in the *organizations* column to a list (they are currently strings because we cannot save Python lists to file within Pandas dataframes, they are automatically converted to strings).

In [9]:
import ast

df['organizations'] = df['organizations'].apply(lambda x: ast.literal_eval(x))

In [73]:
# initialize sentiment dictionary
sentiment = {}

# loop through dataframe and extract org labels and sentiment scores into sentiment dictionary
for i, row in df.iterrows():
    # extract sentiment direction and score
    direction = row['sentiment'].value
    score = row['sentiment'].score
    # loop through each label in organizations column
    for org in row['organizations']:
        # check if org label exists in sentiment dictionary already
        if org not in sentiment.keys():
            # if it doesn't, initialize new entry in dictionary
            sentiment[org] = {'POSITIVE': [], 'NEGATIVE': []}
        # append positive/negative score to respective dictionary entry
        sentiment[org][direction].append(score)

In [74]:
sentiment['ARK']

{'POSITIVE': [0.99809330701828,
  0.996638298034668,
  0.8169347643852234,
  0.7470282316207886,
  0.999733030796051,
  0.9690123200416565,
  0.9982605576515198],
 'NEGATIVE': [0.9974728226661682,
  0.6304851770401001,
  0.9942750334739685,
  0.9999703168869019,
  0.9816587567329407,
  0.9989243149757385,
  0.9981945157051086,
  0.9999686479568481]}

Now we can loop through each organization entry in the sentiment dictionary and calculate an average positive, and average negative score:

In [75]:
# initialize sentiment list
avg_sentiment = []

# loop through each organization
for org in sentiment.keys():
    # get number of positive and negative ratings
    freq = len(sentiment[org]['POSITIVE']) + len(sentiment[org]['NEGATIVE'])
    for direction in ['POSITIVE', 'NEGATIVE']:
        # assign to variable for cleaner code
        score = sentiment[org][direction]
        # if there are no entries, set to 0
        if len(score) == 0:
            sentiment[org][direction] = 0.0
        else:
            # otherwise calculate total
            sentiment[org][direction] = sum(score)
    # now calculate total amount
    total = sentiment[org]['POSITIVE'] - sentiment[org]['NEGATIVE']
    # and the average score
    avg = total/freq
    # add to sentiment list
    avg_sentiment.append({
        'entity': org,
        'positive': sentiment[org]['POSITIVE'],
        'negative': sentiment[org]['NEGATIVE'],
        'frequency': freq,
        'score': avg
    })

In [78]:
sentiment_df = pd.DataFrame(avg_sentiment)
sentiment_df.head()

,entity,positive,negative,frequency,score
0,Bostic,0.0,0.991645,1,-0.991645
1,St. Louis Fed,0.0,0.991645,1,-0.991645
2,Raphael Bostic,0.0,0.991645,1,-0.991645
3,the Atlanta Fed,0.0,0.991645,1,-0.991645
4,Rebound \n&gt,0.0,0.991645,1,-0.991645


Immediately we can see we have a lot of entities which have appeared once in our dataset, and because of this their score will be pushed to one extreme or the other. We can filter out anything with less than or equal to a frequency of `3` to remove many of these instances:

In [82]:
sentiment_df = sentiment_df[sentiment_df['frequency'] > 3]
sentiment_df

,entity,positive,negative,frequency,score
5,Fed,2.326646,10.933105,14,-0.614747
7,Treasury,0.610764,3.990731,5,-0.675993
8,ARK,6.525701,7.600950,15,-0.071683
11,Citadel,0.901277,2.829939,4,-0.482165
17,eBay,1.879811,2.979899,5,-0.220018
...,...,...,...,...,...
1349,IBM,2.965104,0.883970,4,0.520283
1487,PLTR,1.624318,1.911520,4,-0.071801
1553,LMND,0.000000,4.746248,5,-0.949250
1735,PLUG,0.951824,2.889800,4,-0.484494


Here we have some more relevant information. We can see a few items that we can remove through the `BLACKLIST` covered in earlier sections such as *Fed* and *Treasury*, but nonetheless this list is looking much better than before. We can apply `sort` to search for the entities with the highest overall score:

In [84]:
sentiment_df.sort_values('score', ascending=False).head(10)

,entity,positive,negative,frequency,score
1349,IBM,2.965104,0.883970,4,0.520283
317,TAM,5.415756,1.880311,8,0.441931
2011,Sony,4.888052,1.970413,7,0.416806
287,AR,2.548800,0.999818,4,0.387246
504,Intel,3.268430,1.953941,6,0.219082
48,cannabis,6.215654,3.820414,11,0.217749
908,Verizon,3.618954,2.800269,7,0.116955
307,Google,6.957319,5.516039,14,0.102949
452,Company,7.041967,5.876644,14,0.083237
337,SaaS,5.450681,4.846271,11,0.054946


Very quickly we've got our results that we have pulled together using simple, ready-to-use models, and **zero** text preprocessing. With further fine-tuning, and process development, these already good results can become great. Which we will cover soon.